In [1]:
from __future__ import print_function
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import datasets
from skimage import exposure
import numpy as np
import matplotlib.pyplot as plt

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Load data

In [4]:
MNIST = input_data.read_data_sets("MNIST_data", one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
X_train = np.vstack([img.reshape(-1,) for img in MNIST.train.images])
Y_train = MNIST.train.labels
X_test = np.vstack([img.reshape(-1,) for img in MNIST.test.images])
Y_test = MNIST.test.labels

# Prepare data

In [6]:
#Extract the train data for two digit classes 2 and 3
train_data = {}
train_ind = []
for i in range(len(Y_train)):
    if Y_train[i] == 2 or Y_train[i] == 3:
        train_ind.append(i)
train_data['data'] = [X_train[i] for i in train_ind]
train_data['target'] = [Y_train[i] for i in train_ind]

In [7]:
#Extract the test data for two digit classes 2 and 3
test_data = {}
test_ind = []
for i in range(len(Y_test)):
    if Y_test[i] == 2 or Y_test[i] == 3:
        test_ind.append(i)
test_data['data'] = [X_test[i] for i in test_ind]
test_data['target'] = [Y_test[i] for i in test_ind]

# Logistic Regression

In [8]:
logreg = LogisticRegression(solver = 'lbfgs')
#Train model
logreg.fit(train_data['data'], train_data['target'])
logreg_predictions = logreg.predict(test_data['data'])

In [9]:
#Accuracy score:
logreg.score(test_data['data'], test_data['target'])

0.9789422135161606

In [10]:
cm = metrics.confusion_matrix(test_data['target'], logreg_predictions)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

In [11]:
#The normalized confusion matrix:
cm_normalized

array([[0.97577519, 0.02422481],
       [0.01782178, 0.98217822]])

# K-nearest-neighbor

In [12]:
# Split train data into train and validation set:
(trainData, valData, trainLabels, valLabels) = train_test_split(train_data['data'] , train_data['target'] , test_size=0.1, random_state=84)

In [13]:
# Train our classifier and find the optimal value of k
# initialize the values of k for our k-Nearest Neighbor classifier
# to save time, we can test only odd values of k, instead of testing consecutive k
# create a list to store the accuracies for each value of k
accuracies = []
kVals = range(1, 8, 2)
# loop over various values of `k` for the k-Nearest Neighbor classifier
for k in kVals:
    # train the k-Nearest Neighbor classifier with the current value of `k`
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(trainData, trainLabels)
    # evaluate the model using validation data and update the accuracies list
    score = knn.score(valData, valLabels)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))
    accuracies.append(score)

k=1, accuracy=99.19%
k=3, accuracy=99.55%
k=5, accuracy=99.55%
k=7, accuracy=99.37%


In [14]:
# find the value of k that has the largest accuracy - this is usable in case we considered more values of ks
i = np.argmax(accuracies)
print("k=%d achieved highest accuracy of %.2f%% on validation data" % (kVals[i], accuracies[i] * 100))
# re-train our classifier using the best k value and predict the labels of the
# test data
best_knn = KNeighborsClassifier(n_neighbors=kVals[i])
best_knn.fit(trainData, trainLabels)
predictions = best_knn.predict(test_data['data'])

k=3 achieved highest accuracy of 99.55% on validation data


In [15]:
# Get the accuracy score on the test set
knn_score = knn.score(test_data['data'], test_data['target'])

In [16]:
knn_score

0.9965719882468168

In [17]:
# show a final classification report demonstrating the accuracy of the classifier
# for each of the digits
print("EVALUATION ON TESTING DATA")
print(classification_report(test_data['target'], predictions))

EVALUATION ON TESTING DATA
             precision    recall  f1-score   support

          2       1.00      1.00      1.00      1032
          3       1.00      1.00      1.00      1010

avg / total       1.00      1.00      1.00      2042



# Compare results:
Accuracy scores for the two model:
- k-neighbor-neighbor: 99.66 %
- logistic regression: 97.89 %

Based on the accuracy scores, we can conclude that knn nearest neighbor performs slightly better than logistic regression in classifying "2" and "3" 